# Untied Airlines

Prof. Wright hates flying United airlines through O'Hare (ORD). This is a problem, as he is a sought-after lecturer who frequently must make trips from his home base in Madison (MSN) to San Francisco (SFO), Houston (IAH), Washington DC (DCA), and Orlando (MCO). If Prof. Wright flies United, he must travel through ORD, if he travels Delta he can choose to go via Detriot (DTW) or Minneapolis (MSP).

The travel times between various locations in minutes are:<p>

MSN.ORD 27, MSN.DTW 52, MSN.MSP 46,<br>
MSP.SFO 216, MSP.IAH 146, MSP.DCA 118, MSP.MCO 163,<br>
ORD.SFO 246, ORD.IAH 131, ORD.DCA  83, ORD.MCO 130,<br>
DTW.SFO 275, DTW.IAH 154, DTW.DCA  64, DTW.MCO 127<br>


Delay times at ORD are approximately uniformly distributed between 0 and 2.5 hours,
at DTW the delay times are between 0 and 1.5 hours and between 0 and 2 hours at MSP.  

We will assume that Prof. Wright makes 3 trips to each location every year.  Prof. Wright is a notorious cheapskate, and lives for frequent flyer miles.  So he would like to only fly one airline. 

### Should Prof. Wright switch to Delta?  Justify your answer with a mathematical model and explain what your model does to deal with the uncertainty.  

In [1]:
# Load the gams extension
%load_ext gams_magic

__The GAMS Model:__  Explain in this markdown box how you are treating uncertainty

In [2]:
%%gams

set airport /MSN, ORD, DTW, MSP, SFO, IAH, DCA, MCO/;
set home(airport) /MSN/;
set hub(airport) /ORD, DTW, MSP/;
set destination(airport) /SFO,IAH,DCA,MCO/;

alias (airport,a,aa)

set travel(airport,airport)
/MSN.ORD, MSN.DTW, MSN.MSP,
MSP.SFO, MSP.IAH, MSP.DCA, MSP.MCO,
ORD.SFO, ORD.IAH, ORD.DCA, ORD.MCO,
DTW.SFO, DTW.IAH, DTW.DCA, DTW.MCO/;

parameter time1(home,hub)
/MSN.ORD 27, MSN.DTW 52, MSN.MSP 46/;

parameter time2(hub,destination)
/MSP.SFO 216, MSP.IAH 146, MSP.DCA 118, MSP.MCO 163,
ORD.SFO 246, ORD.IAH 131, ORD.DCA 83, ORD.MCO 130,
DTW.SFO 275, DTW.IAH 154, DTW.DCA 64, DTW.MCO 127/;

set delaylim /delaymin, delayMax/;

* I take the mean of the uniform delays
parameter delay(hub)
    /ORD 1.25,
     DTW 0.75,
     MSP 1.0/
;

parameter 
    s(home) Supply
    d(destination) Demand;

variables
    flow(home,hub) Flow on travel a-aa
    flow2(hub,destination) Flow on travel a-aa    
    COST Total cost of assigned flows;

equations supply, demand, conservation, totalcost;

supply(home).. sum(hub, flow(home,hub)) =E= s(home);
demand(destination).. sum(hub, flow2(hub,destination)) =E= d(destination);
conservation(hub).. sum(home, flow(home,hub)) =E= sum(destination, flow2(hub,destination));
totalcost.. COST =e= sum((home,hub), time1(home,hub)*flow(home,hub))
                    + sum((hub,destination), time2(hub,destination)*flow2(hub,destination))
                    + sum((hub,destination), 60*delay(hub)*flow2(hub,destination) );

model untied /supply, demand, conservation, totalcost/;

s(home) = 12;
d(destination) = 3;

scalar sumation /0/;
parameter flows(*,*,*);

* PART 1 ******************************
*$ONTEXT
parameter Soln(*);
set airline /United, Delta/;
sumation = 0;
loop(airline,
    flow.LO(home,hub) = 0; 
    flow.UP(home,hub) = +inf;
    flow.l(home,hub) = 0;
    flow2.LO(hub,destination) = 0; 
    flow2.UP(hub,destination) = +inf;
    flow2.l(hub,destination) = 0;
    if((ord(airline)<2),
        flow.UP(home,'DTW') = 0;
        flow.UP(home,'MSP') = 0;
        );
    if((ord(airline)>1),
        flow.UP(home,'ORD') = 0;
        );
    solve untied using lp minimizing cost;
    sumation = sumation + cost.L;
    display flow.l;
    display flow2.l;

    flows(airline,home,hub) = flow.l(home,hub); 
    flows(airline,hub,destination) = flow2.l(hub,destination);

    display sumation;
    Soln(airline) = sumation;
    sumation = 0;
);
display Soln;
option flows:0:2:1; display flows;
*$OFFTEXT


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),2994.0,9,16,LP,CPLEX,0
1,Normal (1),Optimal Global (1),2874.0,9,16,LP,CPLEX,0


In [3]:
%gams_pull -d soln
print(soln)
soln.set_index('*',inplace=True)
print('Frequent flyer program is ', soln['value'].idxmin())

        *   value
0  United  2994.0
1   Delta  2874.0
Frequent flyer program is  Delta


### What if you add a constraint that he must always use the same hub?

In [7]:
%%gams

* PART 2 ******************************
*$ONTEXT
parameter Soln2(*);
set hubs(airport) /ORD, DTW, MSP/;
sumation = 0;
loop(hubs,
    flow.LO(home,hub) = 0; 
    flow.UP(home,hub) = +inf;
    flow.l(home,hub) = 0;
    flow2.LO(hub,destination) = 0; 
    flow2.UP(hub,destination) = +inf;
    flow2.l(hub,destination) = 0;
    if((ord(hubs)<2),
        flow.UP(home,'DTW') = 0;
        flow.UP(home,'MSP') = 0;      
    );
    if(((ord(hubs)>1) and (ord(hubs)<3)),
        flow.UP(home,'ORD') = 0;
        flow.UP(home,'MSP') = 0;
    );
    if((ord(hubs)>2),
        flow.UP(home,'ORD') = 0;
        flow.UP(home,'DTW') = 0;
    );
    solve untied using lp minimizing cost;        
    sumation = sumation + cost.L;

    flows(hub,home,hub) = flow.l(home,hub); 
    flows(hub,hub,destination) = flow2.l(hub,destination);

    display sumation;
    Soln2(hubs) = sumation;
    sumation = 0;
);
display Soln2;
option flows:0:2:1; display flows;
*$OFFTEXT


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),2994.0,9,16,LP,CPLEX,0
1,Normal (1),Optimal Global (1),3024.0,9,16,LP,CPLEX,0.015
2,Normal (1),Optimal Global (1),3201.0,9,16,LP,CPLEX,0


In [12]:
%gams_pull -d soln2
soln2.set_index('*',inplace=True)
print('Use hub ', soln2['value'].idxmin())

Use hub  ORD


### What if he forgoes frequent flyer miles - which route should he then use for which flight? 

In [10]:
%%gams

* PART 3 ******************************
*$ONTEXT
sumation = 0;
flow.LO(home,hub) = 0; 
flow.UP(home,hub) = +inf;
flow.l(home,hub) = 0;
flow2.LO(hub,destination) = 0; 
flow2.UP(hub,destination) = +inf;
flow2.l(hub,destination) = 0;
solve untied using lp minimizing cost;        
sumation = sumation + cost.L;

flows("noff",home,hub) = flow.l(home,hub); 
flows("noff",hub,destination) = flow2.l(hub,destination);

display sumation;
Soln("noff") = sumation;
display Soln;
option flows:0:2:1; display flows;


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),2820.0,9,16,LP,CPLEX,0.016


__Pull the objective values from GAMS and fastest route time:__

In [11]:
%gams_pull -d soln flow
soln.set_index('*',inplace=True)
print('Fastest route time(mins) ', soln.loc['noff','value'])
display(flow.loc[flow['level'] > 0])

Fastest route time(mins)  2820.0


,home,hub,level,marginal,lower,upper,scale
0,MSN,ORD,3.0,0.0,0.0,inf,1.0
1,MSN,DTW,6.0,0.0,0.0,inf,1.0
2,MSN,MSP,3.0,0.0,0.0,inf,1.0
